In [3]:
import pandas as pd
import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 약간의 전처리 과정

In [149]:
df = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/first_model_data.csv')
temp = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/youtube.csv')

In [150]:
# 시작과 끝을 분리해준다.
week = temp['주간'].str.split('-')
# 원래 데이터와 형태 맞춰주기 위해
# 그리고 유튜브 주기랑 가온차트 주기가 다르다. 
# 5를 빼주면 가온차트와 같아진다. 
start = []
end = []
for i in week:
    start.append(str(datetime.datetime.strptime(i[0],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])
    end.append(str(datetime.datetime.strptime(i[1],'%Y%m%d') - datetime.timedelta(days = 5))[0:10])

In [151]:
# 전처리한 값 추가
temp['start'] = start
temp['end'] = end

In [152]:
# song과 날짜로 합칠거라서 song 전처리 해준다 
# 이유: 이름이 다른 사람들이 존재 ex) 조정석 == Jo Jung-suk
song = []
for i in temp['song']:
    a = re.split('[(-)]+' ,i)[0]
    song.append(a.strip())

In [153]:
temp['song'] = song
temp.head()

,주간,순위,song,name,lastweek_rank,지속기간,변경,조회수,start,end
0,20200417-20200423,1위,WANNABE,ITZY,지난주 #1,7 weeks,-9.5%,2.48M,2020-04-12,2020-04-18
1,20200417-20200423,2위,덤더럼,에이핑크,지난주 #4,2 weeks,30.1%,2.1M,2020-04-12,2020-04-18
2,20200417-20200423,3위,돌덩이,하현우,지난주 #2,9 weeks,-16.4%,1.66M,2020-04-12,2020-04-18
3,20200417-20200423,4위,시작,가호,지난주 #5,11 weeks,-10.6%,1.38M,2020-04-12,2020-04-18
4,20200417-20200423,5위,아로하,Jo Jung-suk,지난주 #11,4 weeks,26.7%,1.32M,2020-04-12,2020-04-18


In [154]:
# 필요없는 값 제거
df_for_merge = temp.drop(columns = ['주간','lastweek_rank','지속기간','변경','name'])

In [155]:
# 이름 변경
df_you = df_for_merge.rename(columns = {'start':'st_day', 'end':'ed_day','song':'name'})

In [156]:
df_you['st_day']

0        2020-04-12
1        2020-04-12
2        2020-04-12
3        2020-04-12
4        2020-04-12
            ...    
10395    2018-04-22
10396    2018-04-22
10397    2018-04-22
10398    2018-04-22
10399    2018-04-22
Name: st_day, Length: 10400, dtype: object

In [157]:
a = []
for i in df_you['st_day']:
    a.append(i.split('-'))

In [158]:
year = []
day = []
month = []
for i in range(0,len(a)):
    years = int(a[i][0][2:4])
    months = int(a[i][1])
    days = int(a[i][2])
    year.append(years)
    month.append(months)
    day.append(days)

In [159]:
df_you['year'] = year
df_you['month'] = month
df_you['day'] = day

In [160]:
df_you = df_you.drop(columns = ['st_day','ed_day'])

In [161]:
순위 = []
조회수 = []
# 조회수 전처리
for i in df_you['조회수']:
    if i == 0:
        조회수.append(0)
    elif i != 0:
        if 'M' in i:
            b = i.replace('M','00000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))
        elif 'K' in i:
            b = i.replace('K','0000')
            if '.' in b:
                b = b.replace('.','')
            조회수.append(int(b))

# 순위 전처리
for i in df_you['순위']:
    if i == 0:
        순위.append(i)
    elif i != 0:
        i = i.replace('위','')
        순위.append(int(i))

In [162]:
df_you['순위'] = 순위
df_you['조회수'] = 조회수

In [163]:
df_you

,순위,name,조회수,year,month,day
0,1,WANNABE,24800000,20,4,12
1,2,덤더럼,2100000,20,4,12
2,3,돌덩이,16600000,20,4,12
3,4,시작,13800000,20,4,12
4,5,아로하,13200000,20,4,12
...,...,...,...,...,...,...
10395,96,야생화,1660000,18,4,22
10396,97,북,1650000,18,4,22
10397,98,DINOSAUR,1630000,18,4,22
10398,99,"Not Spring, Love, or Cherry Blossoms",1620000,18,4,22


In [164]:
# 유튜브와 합치기
a = pd.merge(df,df_you,on=['name','year','month','day'],how = 'left')

In [165]:
a

,Unnamed: 0,artist,name,previous_song_rank,st_day,X1,id,production,distributor,ed_day,...,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수
0,1,(여자)아이들,LATATA,NaN,2018.5.6,235,235,큐브엔터테인먼트,카카오 M,2018.5.12,...,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0
1,2,(여자)아이들,LATATA,NaN,2018.5.13,319,319,큐브엔터테인먼트,카카오 M,2018.5.19,...,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN
2,3,(여자)아이들,LATATA,NaN,2018.5.20,416,416,큐브엔터테인먼트,카카오 M,2018.5.26,...,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN
3,4,(여자)아이들,LATATA,NaN,2018.5.27,512,512,큐브엔터테인먼트,카카오 M,2018.6.2,...,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN
4,5,(여자)아이들,LATATA,NaN,2018.6.3,614,614,큐브엔터테인먼트,카카오 M,2018.6.9,...,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,17396,히키,사랑에 연습이 있었다면,NaN,2019.6.23,8701,8701,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,...,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN
17479,17397,히키,사랑에 연습이 있었다면,NaN,2019.6.30,8904,8904,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,...,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN
17480,17398,히키,사랑에 연습이 있었다면,NaN,2019.7.7,9141,9141,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,...,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN
17481,17399,히키,사랑에 연습이 있었다면,NaN,2019.7.14,9364,9364,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,...,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN


In [166]:
df = a.drop_duplicates(['artist','name','st_day','ed_day'],keep='last')

In [167]:
a = []
for i in list(df['장르']):
    b = i.split(' / ')
    a.append(b)
    
genre = []
for i in range(0,len(list(a))):
    genre.append(a[i][1])

df['genre'] = genre

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [168]:
df

,Unnamed: 0,artist,name,previous_song_rank,st_day,X1,id,production,distributor,ed_day,...,dc_m_recommend,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수
0,1,(여자)아이들,LATATA,NaN,2018.5.6,235,235,큐브엔터테인먼트,카카오 M,2018.5.12,...,0.0,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0
1,2,(여자)아이들,LATATA,NaN,2018.5.13,319,319,큐브엔터테인먼트,카카오 M,2018.5.19,...,0.0,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN
2,3,(여자)아이들,LATATA,NaN,2018.5.20,416,416,큐브엔터테인먼트,카카오 M,2018.5.26,...,0.0,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN
3,4,(여자)아이들,LATATA,NaN,2018.5.27,512,512,큐브엔터테인먼트,카카오 M,2018.6.2,...,0.0,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN
4,5,(여자)아이들,LATATA,NaN,2018.6.3,614,614,큐브엔터테인먼트,카카오 M,2018.6.9,...,0.0,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17478,17396,히키,사랑에 연습이 있었다면,NaN,2019.6.23,8701,8701,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,...,0.0,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN
17479,17397,히키,사랑에 연습이 있었다면,NaN,2019.6.30,8904,8904,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,...,0.0,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN
17480,17398,히키,사랑에 연습이 있었다면,NaN,2019.7.7,9141,9141,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,...,0.0,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN
17481,17399,히키,사랑에 연습이 있었다면,NaN,2019.7.14,9364,9364,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,...,0.0,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN


In [171]:
df.to_csv('basic.csv')

In [172]:
df = df.drop(columns = ['X1','id','Unnamed: 0','name_artist','song_id','st_day','production','distributor','ed_day','장르','런타임','성별','score','rank','활동유형','wks','week','active_g','top_freq_g'])

In [173]:
df = df.drop(columns = ['dc_t_number','dc_t_recommend','dc_t_views','dc_m_recommend','runtime_g','dc_m_views','active'])

In [174]:
pd.set_option('display.max_columns', 999)

In [175]:
df.loc[df['previous_song_rank'].isnull(),'previous_song_rank'] = 200

In [176]:
df.loc[df['순위'].isnull(),'순위'] = 100
df.loc[df['조회수'].isnull(),'조회수'] = 0

In [177]:
df.loc[df['ndc_t_numb'].isnull(),'ndc_t_numb'] = 0
df.loc[df['ndc_t_rec'].isnull(),'ndc_t_rec'] = 0
df.loc[df['ndc_t_view'].isnull(),'ndc_t_view'] = 0

In [ ]:
df.to_csv('first_model_glm.csv')

위에는 전처리 과정. 다른 사람들과 협업함.

In [9]:
temp = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/data/production_score.csv')

In [10]:
temp

,Unnamed: 0,id,artist,name,previous_song_rank,st_day,production,distributor,ed_day,score,...,dc_m_views,gg_score,nv_score,ndc_t_numb,ndc_t_rec,ndc_t_view,season,순위,조회수,pd_score
0,1,1,(여자)아이들,LATATA,NaN,2018-05-06,큐브엔터테인먼트,카카오 M,2018.5.12,12149754,...,0.0,59.0,64.3,NaN,NaN,NaN,spring,4.0,19600000.0,5.6
1,2,2,(여자)아이들,LATATA,NaN,2018-05-13,큐브엔터테인먼트,카카오 M,2018.5.19,19604420,...,0.0,59.0,34.9,NaN,NaN,NaN,spring,NaN,NaN,3.9
2,3,3,(여자)아이들,LATATA,NaN,2018-05-20,큐브엔터테인먼트,카카오 M,2018.5.26,19342281,...,0.0,23.0,38.1,NaN,NaN,NaN,spring,NaN,NaN,3.8
3,4,4,(여자)아이들,LATATA,NaN,2018-05-27,큐브엔터테인먼트,카카오 M,2018.6.2,21249155,...,0.0,40.0,23.2,NaN,NaN,NaN,spring,NaN,NaN,3.4
4,5,5,(여자)아이들,LATATA,NaN,2018-06-03,큐브엔터테인먼트,카카오 M,2018.6.9,19646611,...,0.0,60.0,30.3,NaN,NaN,NaN,summer,NaN,NaN,3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,17359,17479,히키,사랑에 연습이 있었다면,NaN,2019-06-23,"꿈의엔진, 디원미디어",카카오 M,2019.6.29,6356941,...,0.0,84.0,36.6,NaN,NaN,NaN,summer,NaN,NaN,0.0
17359,17360,17480,히키,사랑에 연습이 있었다면,NaN,2019-06-30,"꿈의엔진, 디원미디어",카카오 M,2019.7.6,6905082,...,0.0,32.0,32.7,NaN,NaN,NaN,summer,NaN,NaN,0.0
17360,17361,17481,히키,사랑에 연습이 있었다면,NaN,2019-07-07,"꿈의엔진, 디원미디어",카카오 M,2019.7.13,4389674,...,0.0,31.0,25.1,NaN,NaN,NaN,summer,NaN,NaN,0.0
17361,17362,17482,히키,사랑에 연습이 있었다면,NaN,2019-07-14,"꿈의엔진, 디원미디어",카카오 M,2019.7.20,3547026,...,0.0,54.0,21.1,NaN,NaN,NaN,summer,NaN,NaN,0.0


In [46]:
df = pd.read_csv('/Users/gunwook/Desktop/git_folder/데이터사이언스입문/코드/log_scaled_data.csv')

In [47]:
df.head()

,rank_g,you_rank_g,previous_ranking,genre,season,sex,song_type,active_type,runtime,top_freq,gg_score,nv_score,total_view,season_genre_score,pd_score,dc_total_numb,dc_mean_reccomend,dc_mean_views,drama_view
0,4,2,4,댄스,summer,female,K_POP,Group,0.523669,0.221154,0.80,1.000,0.090654,0.394525,0.056,0.0,0.0,0.0,0.0
1,2,10,2,댄스,summer,female,K_POP,Group,0.523669,0.221154,0.60,0.303,0.000000,0.394525,0.039,0.0,0.0,0.0,0.0
2,10,10,10,댄스,winter,female,K_POP,Group,0.523669,0.221154,0.25,0.073,0.000000,0.696854,0.038,0.0,0.0,0.0,0.0
3,2,10,2,댄스,summer,female,K_POP,Group,0.523669,0.221154,0.20,0.092,0.000000,0.394525,0.034,0.0,0.0,0.0,0.0
4,8,4,7,댄스,fall,female,K_POP,Group,0.523669,0.221154,0.46,0.087,0.045587,0.503168,0.032,0.0,0.0,0.0,0.0


In [ ]:
df = df.drop_duplicates(['artist','name','sex','ymd'],keep='last')

In [13]:
df = df.reset_index()

In [14]:
del df['index']

In [15]:
df['pd_score'] = temp['pd_score']

In [16]:
df = df.drop(columns = ['artist','name','year' ,'month' ,'day'])

In [17]:
del df['X1']

In [18]:
del df['X1_1']

In [19]:
del df['you_rank']

In [20]:
del df['previous_song_rank_g']

In [21]:
del df['ymd']

In [22]:
df = df.rename(columns = {'조회수':'total_view'})

In [23]:
df.loc[df['previous_ranking'].isnull(),'previous_ranking'] = '10'

In [24]:
df.to_csv('new_model_pred.csv')

In [49]:
df = pd.get_dummies(df, columns = ['sex'])
df = pd.get_dummies(df, columns = ['song_type'])
df = pd.get_dummies(df, columns = ['you_rank_g'])

In [50]:
df

,rank_g,previous_ranking,genre,season,active_type,runtime,top_freq,gg_score,nv_score,total_view,...,you_rank_g_1,you_rank_g_2,you_rank_g_3,you_rank_g_4,you_rank_g_5,you_rank_g_6,you_rank_g_7,you_rank_g_8,you_rank_g_9,you_rank_g_10
0,4,4,댄스,summer,Group,0.523669,0.221154,0.80,1.000,0.090654,...,0,1,0,0,0,0,0,0,0,0
1,2,2,댄스,summer,Group,0.523669,0.221154,0.60,0.303,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,10,10,댄스,winter,Group,0.523669,0.221154,0.25,0.073,0.000000,...,0,0,0,0,0,0,0,0,0,1
3,2,2,댄스,summer,Group,0.523669,0.221154,0.20,0.092,0.000000,...,0,0,0,0,0,0,0,0,0,1
4,8,7,댄스,fall,Group,0.523669,0.221154,0.46,0.087,0.045587,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17358,10,1,인디,summer,Solo,0.674556,0.067308,0.31,0.251,0.000000,...,0,0,0,0,0,0,0,0,0,1
17359,9,1,인디,summer,Solo,0.674556,0.067308,0.41,1.000,0.000000,...,0,0,0,0,0,0,0,0,0,1
17360,10,1,인디,summer,Solo,0.674556,0.067308,0.41,0.460,0.000000,...,0,0,0,0,0,0,0,0,0,1
17361,10,1,인디,summer,Solo,0.674556,0.067308,0.22,0.165,0.000000,...,0,0,0,0,0,0,0,0,0,1


In [51]:
df = pd.get_dummies(df, columns = ['season'])

In [52]:
df = pd.get_dummies(df, columns = ['genre'])

In [53]:
df = pd.get_dummies(df, columns = ['active_type'])

In [54]:
df = pd.get_dummies(df, columns = ['previous_ranking'])

In [55]:
from sklearn.model_selection import train_test_split

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ['rank_g']),df['rank_g'], test_size = 0.3)

# 사용 예정 모델
- XGboost
- LightGBM
- RandomForest
- Catboost
- Logistic regression

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Logistic Regression

In [58]:
model = LogisticRegression(random_state=1)

In [59]:
stf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)

In [60]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')
score.mean()

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-pa

0.6997561050303264

In [61]:
model.fit(X_train,y_train)

/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/gunwook/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [62]:
pred_y = model.predict(X_test)

In [63]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.689


# RandomForest

In [64]:
model = RandomForestClassifier(random_state =1, n_estimators = 500)

In [65]:
stf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1)

In [66]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [67]:
score.mean()

0.6888141663812283

In [68]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [69]:
pred_y = model.predict(X_test)

In [70]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.679


# Catboost

In [71]:
from catboost import CatBoostClassifier

In [72]:
model = CatBoostClassifier(n_estimators = 1000, learning_rate =0.05)

In [73]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

0:	learn: 2.1218691	total: 76.4ms	remaining: 1m 16s
1:	learn: 1.9905859	total: 104ms	remaining: 51.8s
2:	learn: 1.8885932	total: 127ms	remaining: 42.1s
3:	learn: 1.8087862	total: 146ms	remaining: 36.3s
4:	learn: 1.7311262	total: 165ms	remaining: 32.9s
5:	learn: 1.6822484	total: 185ms	remaining: 30.7s
6:	learn: 1.6348061	total: 206ms	remaining: 29.2s
7:	learn: 1.5873728	total: 226ms	remaining: 28s
8:	learn: 1.5456824	total: 246ms	remaining: 27s
9:	learn: 1.5047058	total: 267ms	remaining: 26.4s
10:	learn: 1.4692269	total: 286ms	remaining: 25.7s
11:	learn: 1.4363141	total: 305ms	remaining: 25.1s
12:	learn: 1.4060294	total: 322ms	remaining: 24.5s
13:	learn: 1.3818413	total: 339ms	remaining: 23.9s
14:	learn: 1.3536517	total: 356ms	remaining: 23.4s
15:	learn: 1.3254703	total: 376ms	remaining: 23.1s
16:	learn: 1.3014290	total: 395ms	remaining: 22.9s
17:	learn: 1.2834565	total: 417ms	remaining: 22.7s
18:	learn: 1.2600039	total: 436ms	remaining: 22.5s
19:	learn: 1.2384899	total: 456ms	remaining

166:	learn: 0.7944300	total: 3.37s	remaining: 16.8s
167:	learn: 0.7933512	total: 3.39s	remaining: 16.8s
168:	learn: 0.7926558	total: 3.41s	remaining: 16.8s
169:	learn: 0.7918924	total: 3.42s	remaining: 16.7s
170:	learn: 0.7912814	total: 3.44s	remaining: 16.7s
171:	learn: 0.7908476	total: 3.46s	remaining: 16.7s
172:	learn: 0.7901275	total: 3.49s	remaining: 16.7s
173:	learn: 0.7895711	total: 3.51s	remaining: 16.7s
174:	learn: 0.7890879	total: 3.53s	remaining: 16.6s
175:	learn: 0.7884603	total: 3.55s	remaining: 16.6s
176:	learn: 0.7876209	total: 3.57s	remaining: 16.6s
177:	learn: 0.7870528	total: 3.6s	remaining: 16.6s
178:	learn: 0.7865395	total: 3.63s	remaining: 16.6s
179:	learn: 0.7857161	total: 3.65s	remaining: 16.6s
180:	learn: 0.7849464	total: 3.67s	remaining: 16.6s
181:	learn: 0.7840046	total: 3.69s	remaining: 16.6s
182:	learn: 0.7835088	total: 3.71s	remaining: 16.6s
183:	learn: 0.7826675	total: 3.73s	remaining: 16.6s
184:	learn: 0.7820639	total: 3.75s	remaining: 16.5s
185:	learn: 0

331:	learn: 0.7021301	total: 6.92s	remaining: 13.9s
332:	learn: 0.7017462	total: 6.95s	remaining: 13.9s
333:	learn: 0.7014431	total: 6.97s	remaining: 13.9s
334:	learn: 0.7011803	total: 7s	remaining: 13.9s
335:	learn: 0.7007826	total: 7.02s	remaining: 13.9s
336:	learn: 0.7004177	total: 7.04s	remaining: 13.8s
337:	learn: 0.7000436	total: 7.06s	remaining: 13.8s
338:	learn: 0.6998318	total: 7.08s	remaining: 13.8s
339:	learn: 0.6992618	total: 7.09s	remaining: 13.8s
340:	learn: 0.6989266	total: 7.11s	remaining: 13.7s
341:	learn: 0.6983698	total: 7.13s	remaining: 13.7s
342:	learn: 0.6978001	total: 7.15s	remaining: 13.7s
343:	learn: 0.6971856	total: 7.17s	remaining: 13.7s
344:	learn: 0.6968166	total: 7.19s	remaining: 13.7s
345:	learn: 0.6961149	total: 7.21s	remaining: 13.6s
346:	learn: 0.6958897	total: 7.23s	remaining: 13.6s
347:	learn: 0.6956556	total: 7.25s	remaining: 13.6s
348:	learn: 0.6952545	total: 7.27s	remaining: 13.6s
349:	learn: 0.6947683	total: 7.28s	remaining: 13.5s
350:	learn: 0.6

499:	learn: 0.6411615	total: 10.5s	remaining: 10.5s
500:	learn: 0.6408512	total: 10.5s	remaining: 10.5s
501:	learn: 0.6406489	total: 10.5s	remaining: 10.4s
502:	learn: 0.6404231	total: 10.6s	remaining: 10.4s
503:	learn: 0.6401921	total: 10.6s	remaining: 10.4s
504:	learn: 0.6399926	total: 10.6s	remaining: 10.4s
505:	learn: 0.6398222	total: 10.6s	remaining: 10.4s
506:	learn: 0.6393641	total: 10.6s	remaining: 10.3s
507:	learn: 0.6391467	total: 10.7s	remaining: 10.3s
508:	learn: 0.6385910	total: 10.7s	remaining: 10.3s
509:	learn: 0.6383714	total: 10.7s	remaining: 10.3s
510:	learn: 0.6380641	total: 10.7s	remaining: 10.3s
511:	learn: 0.6376969	total: 10.8s	remaining: 10.3s
512:	learn: 0.6374543	total: 10.8s	remaining: 10.2s
513:	learn: 0.6371951	total: 10.8s	remaining: 10.2s
514:	learn: 0.6369655	total: 10.8s	remaining: 10.2s
515:	learn: 0.6367031	total: 10.9s	remaining: 10.2s
516:	learn: 0.6361659	total: 10.9s	remaining: 10.2s
517:	learn: 0.6359045	total: 10.9s	remaining: 10.2s
518:	learn: 

664:	learn: 0.5934261	total: 14.7s	remaining: 7.41s
665:	learn: 0.5930585	total: 14.7s	remaining: 7.39s
666:	learn: 0.5926985	total: 14.8s	remaining: 7.37s
667:	learn: 0.5924518	total: 14.8s	remaining: 7.35s
668:	learn: 0.5922773	total: 14.8s	remaining: 7.33s
669:	learn: 0.5917932	total: 14.9s	remaining: 7.31s
670:	learn: 0.5915140	total: 14.9s	remaining: 7.29s
671:	learn: 0.5912838	total: 14.9s	remaining: 7.27s
672:	learn: 0.5911564	total: 14.9s	remaining: 7.25s
673:	learn: 0.5909486	total: 14.9s	remaining: 7.23s
674:	learn: 0.5906424	total: 15s	remaining: 7.21s
675:	learn: 0.5903036	total: 15s	remaining: 7.18s
676:	learn: 0.5900013	total: 15s	remaining: 7.16s
677:	learn: 0.5897617	total: 15s	remaining: 7.14s
678:	learn: 0.5893846	total: 15.1s	remaining: 7.12s
679:	learn: 0.5889827	total: 15.1s	remaining: 7.09s
680:	learn: 0.5887030	total: 15.1s	remaining: 7.07s
681:	learn: 0.5883035	total: 15.1s	remaining: 7.05s
682:	learn: 0.5880066	total: 15.1s	remaining: 7.03s
683:	learn: 0.587811

...

232:	learn: 0.7470070	total: 4.63s	remaining: 15.2s
233:	learn: 0.7463111	total: 4.65s	remaining: 15.2s
234:	learn: 0.7458800	total: 4.67s	remaining: 15.2s
235:	learn: 0.7451682	total: 4.69s	remaining: 15.2s
236:	learn: 0.7444987	total: 4.71s	remaining: 15.2s
237:	learn: 0.7439943	total: 4.73s	remaining: 15.1s
238:	learn: 0.7433865	total: 4.75s	remaining: 15.1s
239:	learn: 0.7427444	total: 4.77s	remaining: 15.1s
240:	learn: 0.7419337	total: 4.79s	remaining: 15.1s
241:	learn: 0.7415188	total: 4.8s	remaining: 15s
242:	learn: 0.7408495	total: 4.82s	remaining: 15s
243:	learn: 0.7403135	total: 4.84s	remaining: 15s
244:	learn: 0.7397599	total: 4.86s	remaining: 15s
245:	learn: 0.7394018	total: 4.88s	remaining: 14.9s
246:	learn: 0.7388848	total: 4.9s	remaining: 14.9s
247:	learn: 0.7382733	total: 4.91s	remaining: 14.9s
248:	learn: 0.7376435	total: 4.93s	remaining: 14.9s
249:	learn: 0.7372624	total: 4.95s	remaining: 14.8s
250:	learn: 0.7367408	total: 4.97s	remaining: 14.8s
251:	learn: 0.7360043	

394:	learn: 0.6764730	total: 7.74s	remaining: 11.9s
395:	learn: 0.6760711	total: 7.76s	remaining: 11.8s
396:	learn: 0.6754928	total: 7.78s	remaining: 11.8s
397:	learn: 0.6752407	total: 7.79s	remaining: 11.8s
398:	learn: 0.6749901	total: 7.81s	remaining: 11.8s
399:	learn: 0.6743907	total: 7.83s	remaining: 11.7s
400:	learn: 0.6739622	total: 7.85s	remaining: 11.7s
401:	learn: 0.6736162	total: 7.87s	remaining: 11.7s
402:	learn: 0.6733257	total: 7.89s	remaining: 11.7s
403:	learn: 0.6729052	total: 7.9s	remaining: 11.7s
404:	learn: 0.6724732	total: 7.92s	remaining: 11.6s
405:	learn: 0.6718074	total: 7.94s	remaining: 11.6s
406:	learn: 0.6713999	total: 7.96s	remaining: 11.6s
407:	learn: 0.6713215	total: 7.98s	remaining: 11.6s
408:	learn: 0.6710673	total: 8s	remaining: 11.6s
409:	learn: 0.6708261	total: 8.01s	remaining: 11.5s
410:	learn: 0.6705312	total: 8.03s	remaining: 11.5s
411:	learn: 0.6701351	total: 8.05s	remaining: 11.5s
412:	learn: 0.6698218	total: 8.07s	remaining: 11.5s
413:	learn: 0.66

555:	learn: 0.6219474	total: 11.1s	remaining: 8.86s
556:	learn: 0.6214584	total: 11.1s	remaining: 8.84s
557:	learn: 0.6211763	total: 11.1s	remaining: 8.82s
558:	learn: 0.6208848	total: 11.2s	remaining: 8.8s
559:	learn: 0.6207364	total: 11.2s	remaining: 8.78s
560:	learn: 0.6204887	total: 11.2s	remaining: 8.77s
561:	learn: 0.6202867	total: 11.2s	remaining: 8.75s
562:	learn: 0.6200322	total: 11.2s	remaining: 8.73s
563:	learn: 0.6199191	total: 11.3s	remaining: 8.71s
564:	learn: 0.6196652	total: 11.3s	remaining: 8.69s
565:	learn: 0.6194391	total: 11.3s	remaining: 8.66s
566:	learn: 0.6193311	total: 11.3s	remaining: 8.64s
567:	learn: 0.6190878	total: 11.3s	remaining: 8.63s
568:	learn: 0.6186638	total: 11.4s	remaining: 8.6s
569:	learn: 0.6184362	total: 11.4s	remaining: 8.58s
570:	learn: 0.6182384	total: 11.4s	remaining: 8.56s
571:	learn: 0.6179522	total: 11.4s	remaining: 8.54s
572:	learn: 0.6177158	total: 11.4s	remaining: 8.52s
573:	learn: 0.6174784	total: 11.4s	remaining: 8.5s
574:	learn: 0.6

723:	learn: 0.5772162	total: 14.5s	remaining: 5.51s
724:	learn: 0.5770173	total: 14.5s	remaining: 5.49s
725:	learn: 0.5768181	total: 14.5s	remaining: 5.47s
726:	learn: 0.5765321	total: 14.5s	remaining: 5.45s
727:	learn: 0.5762802	total: 14.5s	remaining: 5.43s
728:	learn: 0.5760428	total: 14.6s	remaining: 5.42s
729:	learn: 0.5757641	total: 14.6s	remaining: 5.4s
730:	learn: 0.5755721	total: 14.6s	remaining: 5.38s
731:	learn: 0.5753952	total: 14.6s	remaining: 5.35s
732:	learn: 0.5753019	total: 14.6s	remaining: 5.33s
733:	learn: 0.5752036	total: 14.7s	remaining: 5.31s
734:	learn: 0.5749152	total: 14.7s	remaining: 5.29s
735:	learn: 0.5745512	total: 14.7s	remaining: 5.27s
736:	learn: 0.5742498	total: 14.7s	remaining: 5.25s
737:	learn: 0.5739260	total: 14.7s	remaining: 5.23s
738:	learn: 0.5736514	total: 14.8s	remaining: 5.21s
739:	learn: 0.5732815	total: 14.8s	remaining: 5.19s
740:	learn: 0.5731108	total: 14.8s	remaining: 5.17s
741:	learn: 0.5729177	total: 14.8s	remaining: 5.15s
742:	learn: 0

886:	learn: 0.5371710	total: 17.6s	remaining: 2.24s
887:	learn: 0.5367905	total: 17.6s	remaining: 2.22s
888:	learn: 0.5366679	total: 17.6s	remaining: 2.2s
889:	learn: 0.5363985	total: 17.6s	remaining: 2.18s
890:	learn: 0.5362798	total: 17.7s	remaining: 2.16s
891:	learn: 0.5360777	total: 17.7s	remaining: 2.14s
892:	learn: 0.5358775	total: 17.7s	remaining: 2.12s
893:	learn: 0.5357063	total: 17.7s	remaining: 2.1s
894:	learn: 0.5355188	total: 17.8s	remaining: 2.08s
895:	learn: 0.5351723	total: 17.8s	remaining: 2.06s
896:	learn: 0.5350156	total: 17.8s	remaining: 2.05s
897:	learn: 0.5348185	total: 17.8s	remaining: 2.03s
898:	learn: 0.5346359	total: 17.9s	remaining: 2.01s
899:	learn: 0.5345177	total: 17.9s	remaining: 1.99s
900:	learn: 0.5343921	total: 17.9s	remaining: 1.97s
901:	learn: 0.5341044	total: 18s	remaining: 1.95s
902:	learn: 0.5339443	total: 18s	remaining: 1.93s
903:	learn: 0.5337216	total: 18s	remaining: 1.91s
904:	learn: 0.5335931	total: 18s	remaining: 1.89s
905:	learn: 0.5332872	

In [74]:
score.mean()

0.7135784589944325

In [75]:
model.fit(X_train,y_train)

0:	learn: 2.1101114	total: 31.8ms	remaining: 31.8s
1:	learn: 1.9792981	total: 55.1ms	remaining: 27.5s
2:	learn: 1.8678216	total: 84.2ms	remaining: 28s
3:	learn: 1.7881070	total: 111ms	remaining: 27.7s
4:	learn: 1.7156629	total: 141ms	remaining: 28.1s
5:	learn: 1.6650697	total: 175ms	remaining: 29s
6:	learn: 1.6170988	total: 215ms	remaining: 30.4s
7:	learn: 1.5641023	total: 248ms	remaining: 30.8s
8:	learn: 1.5213750	total: 272ms	remaining: 29.9s
9:	learn: 1.4824274	total: 292ms	remaining: 28.9s
10:	learn: 1.4498779	total: 315ms	remaining: 28.3s
11:	learn: 1.4091872	total: 339ms	remaining: 27.9s
12:	learn: 1.3804649	total: 359ms	remaining: 27.3s
13:	learn: 1.3562143	total: 378ms	remaining: 26.7s
14:	learn: 1.3294556	total: 401ms	remaining: 26.3s
15:	learn: 1.3047002	total: 429ms	remaining: 26.4s
16:	learn: 1.2800561	total: 461ms	remaining: 26.7s
17:	learn: 1.2627035	total: 498ms	remaining: 27.2s
18:	learn: 1.2398311	total: 525ms	remaining: 27.1s
19:	learn: 1.2191377	total: 546ms	remainin

167:	learn: 0.7946877	total: 3.62s	remaining: 17.9s
168:	learn: 0.7938266	total: 3.63s	remaining: 17.9s
169:	learn: 0.7928133	total: 3.66s	remaining: 17.9s
170:	learn: 0.7919688	total: 3.68s	remaining: 17.9s
171:	learn: 0.7913653	total: 3.7s	remaining: 17.8s
172:	learn: 0.7905063	total: 3.72s	remaining: 17.8s
173:	learn: 0.7898452	total: 3.74s	remaining: 17.8s
174:	learn: 0.7892241	total: 3.76s	remaining: 17.7s
175:	learn: 0.7885111	total: 3.78s	remaining: 17.7s
176:	learn: 0.7876292	total: 3.8s	remaining: 17.7s
177:	learn: 0.7871443	total: 3.82s	remaining: 17.7s
178:	learn: 0.7866307	total: 3.85s	remaining: 17.7s
179:	learn: 0.7859327	total: 3.87s	remaining: 17.6s
180:	learn: 0.7851019	total: 3.88s	remaining: 17.6s
181:	learn: 0.7840303	total: 3.9s	remaining: 17.6s
182:	learn: 0.7837166	total: 3.92s	remaining: 17.5s
183:	learn: 0.7826907	total: 3.94s	remaining: 17.5s
184:	learn: 0.7820706	total: 3.96s	remaining: 17.5s
185:	learn: 0.7813582	total: 3.98s	remaining: 17.4s
186:	learn: 0.7

329:	learn: 0.7074269	total: 6.96s	remaining: 14.1s
330:	learn: 0.7071015	total: 6.97s	remaining: 14.1s
331:	learn: 0.7068183	total: 6.99s	remaining: 14.1s
332:	learn: 0.7062358	total: 7.01s	remaining: 14s
333:	learn: 0.7058706	total: 7.03s	remaining: 14s
334:	learn: 0.7056356	total: 7.04s	remaining: 14s
335:	learn: 0.7054811	total: 7.06s	remaining: 14s
336:	learn: 0.7051564	total: 7.08s	remaining: 13.9s
337:	learn: 0.7046010	total: 7.1s	remaining: 13.9s
338:	learn: 0.7039699	total: 7.12s	remaining: 13.9s
339:	learn: 0.7037242	total: 7.13s	remaining: 13.8s
340:	learn: 0.7033992	total: 7.15s	remaining: 13.8s
341:	learn: 0.7029955	total: 7.17s	remaining: 13.8s
342:	learn: 0.7023496	total: 7.19s	remaining: 13.8s
343:	learn: 0.7021825	total: 7.21s	remaining: 13.7s
344:	learn: 0.7019304	total: 7.23s	remaining: 13.7s
345:	learn: 0.7013663	total: 7.25s	remaining: 13.7s
346:	learn: 0.7010697	total: 7.26s	remaining: 13.7s
347:	learn: 0.7006049	total: 7.28s	remaining: 13.6s
348:	learn: 0.7003598

491:	learn: 0.6528465	total: 10.1s	remaining: 10.5s
492:	learn: 0.6526091	total: 10.2s	remaining: 10.4s
493:	learn: 0.6521604	total: 10.2s	remaining: 10.4s
494:	learn: 0.6518012	total: 10.2s	remaining: 10.4s
495:	learn: 0.6515313	total: 10.2s	remaining: 10.4s
496:	learn: 0.6511799	total: 10.2s	remaining: 10.4s
497:	learn: 0.6510929	total: 10.3s	remaining: 10.4s
498:	learn: 0.6507772	total: 10.3s	remaining: 10.3s
499:	learn: 0.6504435	total: 10.3s	remaining: 10.3s
500:	learn: 0.6502977	total: 10.3s	remaining: 10.3s
501:	learn: 0.6501203	total: 10.4s	remaining: 10.3s
502:	learn: 0.6499561	total: 10.4s	remaining: 10.3s
503:	learn: 0.6498269	total: 10.4s	remaining: 10.2s
504:	learn: 0.6496696	total: 10.4s	remaining: 10.2s
505:	learn: 0.6495386	total: 10.4s	remaining: 10.2s
506:	learn: 0.6493542	total: 10.5s	remaining: 10.2s
507:	learn: 0.6490503	total: 10.5s	remaining: 10.2s
508:	learn: 0.6488229	total: 10.5s	remaining: 10.1s
509:	learn: 0.6485337	total: 10.5s	remaining: 10.1s
510:	learn: 

654:	learn: 0.6084177	total: 14.2s	remaining: 7.48s
655:	learn: 0.6082330	total: 14.2s	remaining: 7.46s
656:	learn: 0.6080062	total: 14.3s	remaining: 7.44s
657:	learn: 0.6076706	total: 14.3s	remaining: 7.42s
658:	learn: 0.6073806	total: 14.3s	remaining: 7.4s
659:	learn: 0.6071809	total: 14.3s	remaining: 7.38s
660:	learn: 0.6069449	total: 14.3s	remaining: 7.36s
661:	learn: 0.6065706	total: 14.4s	remaining: 7.33s
662:	learn: 0.6064302	total: 14.4s	remaining: 7.31s
663:	learn: 0.6061706	total: 14.4s	remaining: 7.29s
664:	learn: 0.6059091	total: 14.4s	remaining: 7.27s
665:	learn: 0.6053966	total: 14.5s	remaining: 7.25s
666:	learn: 0.6052355	total: 14.5s	remaining: 7.22s
667:	learn: 0.6051126	total: 14.5s	remaining: 7.2s
668:	learn: 0.6049117	total: 14.5s	remaining: 7.18s
669:	learn: 0.6047024	total: 14.5s	remaining: 7.17s
670:	learn: 0.6044356	total: 14.6s	remaining: 7.14s
671:	learn: 0.6042231	total: 14.6s	remaining: 7.12s
672:	learn: 0.6040846	total: 14.6s	remaining: 7.11s
673:	learn: 0.

820:	learn: 0.5682825	total: 18s	remaining: 3.92s
821:	learn: 0.5681148	total: 18s	remaining: 3.89s
822:	learn: 0.5678582	total: 18s	remaining: 3.87s
823:	learn: 0.5676170	total: 18s	remaining: 3.85s
824:	learn: 0.5672500	total: 18s	remaining: 3.83s
825:	learn: 0.5670436	total: 18.1s	remaining: 3.81s
826:	learn: 0.5669602	total: 18.1s	remaining: 3.79s
827:	learn: 0.5668138	total: 18.1s	remaining: 3.77s
828:	learn: 0.5664861	total: 18.2s	remaining: 3.75s
829:	learn: 0.5661351	total: 18.2s	remaining: 3.73s
830:	learn: 0.5659453	total: 18.2s	remaining: 3.71s
831:	learn: 0.5655646	total: 18.3s	remaining: 3.69s
832:	learn: 0.5654237	total: 18.3s	remaining: 3.67s
833:	learn: 0.5651862	total: 18.3s	remaining: 3.64s
834:	learn: 0.5649199	total: 18.3s	remaining: 3.62s
835:	learn: 0.5646325	total: 18.4s	remaining: 3.6s
836:	learn: 0.5644481	total: 18.4s	remaining: 3.58s
837:	learn: 0.5642466	total: 18.4s	remaining: 3.56s
838:	learn: 0.5641059	total: 18.4s	remaining: 3.54s
839:	learn: 0.5637175	t

986:	learn: 0.5305920	total: 21.5s	remaining: 284ms
987:	learn: 0.5303741	total: 21.6s	remaining: 262ms
988:	learn: 0.5301212	total: 21.6s	remaining: 240ms
989:	learn: 0.5300424	total: 21.6s	remaining: 218ms
990:	learn: 0.5298140	total: 21.6s	remaining: 196ms
991:	learn: 0.5295765	total: 21.6s	remaining: 175ms
992:	learn: 0.5293801	total: 21.7s	remaining: 153ms
993:	learn: 0.5292298	total: 21.7s	remaining: 131ms
994:	learn: 0.5289988	total: 21.7s	remaining: 109ms
995:	learn: 0.5287597	total: 21.7s	remaining: 87.3ms
996:	learn: 0.5284778	total: 21.8s	remaining: 65.5ms
997:	learn: 0.5283573	total: 21.8s	remaining: 43.6ms
998:	learn: 0.5281139	total: 21.8s	remaining: 21.8ms
999:	learn: 0.5277762	total: 21.8s	remaining: 0us


In [76]:
pred_y = model.predict(X_test)

In [77]:
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.704


# Xgboost

In [78]:
from xgboost import XGBClassifier

In [79]:
# XGB는 생각보다 성능이 낮다... 아마 파라미터 조정을 해야될듯...
model = XGBClassifier(n_estimators = 500,learning_rate = 0.05)

In [80]:
score = cross_val_score(model, X_train, y_train, cv = stf, scoring = 'f1_micro')

In [81]:
score.mean()

0.6954003804672746

In [82]:
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
pred_y = model.predict(X_test)
print("F1 : %.3f" % f1_score(y_test, pred_y, average = 'micro'))

F1 : 0.687
